In [3]:
from IPython.core.display import HTML

In [4]:
words_colors = {
    'plain' : None,
    'red' : ['#FF0000','#FFFFFF'],
    'green' : ['#008800','#FFFFFF'],
    'blue' : ['#0000FF','#FFFFFF'],
    'black' : ['#000000','#FFFFFF']
}

In [9]:
def word2html(word,color=None):
    s = '<span style="border:1px; border-radius:5px; border-style:solid;'
    if (color is not None) and (color in words_colors): 
        s += ' background-color: '+words_colors[color][0]+';'
        s += ' border-color: '+words_colors[color][0]+';'
        s += ' color: '+words_colors[color][1]+';'
    s+= ' padding: 2px; margin: 2px">'
    s += word
    s+= '</span>'
    return s

In [10]:
HTML('Обычный текст и немного '+word2html('красного','red')+', '+ word2html('зеленого','green')+' и '+word2html('синего','blue')+'. А так же еще немного текста чтобы протестировать переносы '+word2html('блоков&nbspтекста','black')+'.')

## Imports

In [56]:
import re
from gensim.corpora import Dictionary
from gensim.models import ldamodel
import pandas as pd
import numpy
import random
import string
from nltk.corpus import stopwords
from IPython.core.display import HTML

C:\Users\PDudukin\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


## Data

In [59]:
texts = list(pd.read_csv('train_task1_latest.csv', encoding='utf-8').paragraph.values)

## Coloring words in text

In [157]:
def ColorWordInText(word_list, text, list_of_colors, param=False):
    if param == 'expandonlycolor' and len(list_of_colors) < len(word_list) and len(list_of_colors)==1:
        list_of_colors = list_of_colors*len(word_list)
        
    for word, color in zip(word_list, list_of_colors):
        text = re.sub(word, word2html(word,color), text)
    return HTML(text)

In [223]:
text = 'Непомерно дорогой кинотеатр, никогда больше не приду туда.'
word_list = ['дорогой', 'никогда', 'приду']
list_of_colors = ['red', 'green', 'blue']

ColorWordInText(word_list, text, list_of_colors)

## LDA Highlighting: Framework

In [64]:
def HighlightWordsInText_LDA(list_of_texts, list_of_colors, num_topics=1, top_words=5):
    stop_words = stopwords.words('russian')
    texts = [[x for x in re.sub("[\W+\-\W+]\W+"," ",text).split(' ') if x.lower() not in stop_words ] for text in [list_of_texts]]
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    numpy.random.seed(1)
    model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=50)

    word_list = []
    colors_list = list_of_colors[:]
    
    for topic in range(num_topics):
        for word_ix in range(top_words):
            word_list.append(model.show_topic(topic, topn=top_words)[word_ix][0])

        if len(colors_list)<top_words*num_topics:
            for c in range(top_words*num_topics-len(colors_list)):
                colors_list.append(words_colors['black'][0])
    
    return ColorWordInText(word_list, list_of_texts, colors_list)

In [67]:
HighlightWordsInText_LDA(texts[random.randint(0,1000)], list_of_colors, 1, 5)

## Highlighting words by topic: Framework

In [86]:
def HighlightWordsInText_ByTopic(list_of_texts, list_of_colors, num_topics=1, top_words=5):
    stop_words = stopwords.words('russian')
    texts = [[x for x in re.sub("[\W+\-\W+]\W+"," ",text).split(' ') if x.lower() not in stop_words ] for text in [list_of_texts]]
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    #numpy.random.seed(1)
    model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=50)

    word_list = []
    colors_list = []
    colors_sample = random.sample(list_of_colors,num_topics)
    for topic in range(num_topics):
        for word_ix in range(top_words):
            word_list.append(model.show_topic(topic, topn=top_words)[word_ix][0])
        
        colors_list.extend([colors_sample[topic]]*top_words)

    word_list = list(set(word_list))
    colors_list = colors_list[:len(word_list)]

    return ColorWordInText(word_list, list_of_texts, colors_list)

In [98]:
HighlightWordsInText_ByTopic(texts[random.randint(0,1000)], list_of_colors, 2, 5)

## Theathres reviews

In [318]:
import json
import codecs
import pandas as pd
from nltk.corpus import stopwords # repeated
from functools import reduce
from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, GlobalMaxPooling1D
from keras.layers import LSTM
from keras import callbacks
from keras_tqdm import TQDMNotebookCallback
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = stopwords.words('russian')

In [319]:
def GetFlatList(list_of_lists):
    return reduce(lambda x,y: x+y, list_of_lists)

In [320]:
with codecs.open("reviews.json", encoding = 'utf-8') as json_file:
    json_data = json.load(json_file)

id = 2
print("\x1b[1;30mReview:\x1b[0m",json_data[id]['text'])
print("\x1b[1;30mRating:\x1b[0m",json_data[id]['review_rating'])

Review: 
Очень хороший кинотеатр. Особенно нравится зал номер 4. Везде чисто. И цены хорошие. Один из немногих минусов - это то, что не всегда попадаются чистые 3D-очки. Их бы надо протирать. 

Rating: 4


In [321]:
reviews = []
ratings = []

for id in range(len(json_data)):
    try:
        reviews.append(json_data[id]['text'])
    except KeyError:
        reviews.append(np.NaN)
    
    try:
        ratings.append(int(json_data[id]['review_rating']))
    except KeyError:
        ratings.append(-1)

In [322]:
data = pd.DataFrame({'raw_review':reviews,'raw_rating':ratings})
data = data[(data.raw_rating>-1)].dropna()
data.head(2)

,raw_rating,raw_review
0,4,"\nСколько стоят билеты на комедию "" Кухня: Пос..."
1,1,"\nПочему все мультфильмы в 3D, с малышами неуд..."


In [323]:
test_size = 0.2
for id in data.index:
    if id<int(data.shape[0]*(1-test_size)):
        data.loc[id,'part'] = 'train'
    else:
        data.loc[id,'part'] = 'test'

In [324]:
import re
for id in data.index:
    try:
        data.loc[id,'text'] = ' '.join(re.split('\W+',data.loc[id,'raw_review'].lower())[1:-1])
    except AttributeError:
        print(id)

In [325]:
data.set_value(0,'tokens',0)
data['tokens'] = data['tokens'].astype(object)
for id in data.index:  
    data.set_value(id,'tokens',[word for word in data.loc[id,'text'].split(' ') if word not in stop_words and word not in ''])

data['rating'] = data['raw_rating'].map(lambda x: 1 if int(x)>3 else -1)
data.head(2)

,raw_rating,raw_review,part,text,tokens,rating
0,4,"\nСколько стоят билеты на комедию "" Кухня: Пос...",train,сколько стоят билеты на комедию кухня последня...,"[сколько, стоят, билеты, комедию, кухня, после...",1
1,1,"\nПочему все мультфильмы в 3D, с малышами неуд...",train,почему все мультфильмы в 3d с малышами неудобн...,"[почему, мультфильмы, 3d, малышами, неудобно, ...",-1


In [326]:
pos = Counter(GetFlatList([x for x in data[data.rating==1]['tokens'].values]))
sorted(pos.items(), key=lambda t: t[1])[-20:-10]

[('просто', 437),
 ('место', 437),
 ('кинотеатра', 445),
 ('зале', 489),
 ('отличный', 565),
 ('фильм', 606),
 ('залы', 621),
 ('билеты', 668),
 ('цены', 725),
 ('удобные', 732)]

In [327]:
neg = Counter(GetFlatList([x for x in data[data.rating==-1]['tokens'].values]))
sorted(neg.items(), key=lambda t: t[1])[-40:-30]

[('15', 319),
 ('места', 321),
 ('т', 322),
 ('смотреть', 323),
 ('20', 323),
 ('деньги', 353),
 ('экран', 353),
 ('5', 359),
 ('билетов', 362),
 ('очки', 365)]

In [328]:
%time pos_list = GetFlatList([x for x in data[data.rating==+1]['tokens'].values])
%time neg_list = GetFlatList([x for x in data[data.rating==-1]['tokens'].values])
                       
%time pos_words_dict = dict(Counter(pos_list))
%time neg_words_dict = dict(Counter(neg_list))

%time pos_to_delete = list({key: pos_words_dict[key] for key in pos_words_dict.keys() if pos_words_dict[key]<3}.keys())          
%time neg_to_delete = list({key: neg_words_dict[key] for key in neg_words_dict.keys() if neg_words_dict[key]<3}.keys())          

%time clean_pos_list = [x for x in pos_list if x not in pos_to_delete]
%time clean_neg_list = [x for x in neg_list if x not in neg_to_delete]

%time pos_words = [x for x in list(set(clean_pos_list)) if x not in list(set(clean_neg_list))]
%time neg_words = [x for x in list(set(clean_neg_list)) if x not in list(set(clean_pos_list))]

Wall time: 5.42 s
Wall time: 7.21 s
Wall time: 26.1 ms
Wall time: 50.1 ms
Wall time: 10 ms
Wall time: 11 ms
Wall time: 1min 45s
Wall time: 2min 45s
Wall time: 1min 12s
Wall time: 1min 14s


In [329]:
print(pos_words[:9])
print(neg_words[:9])

['чавкают', 'свежо', 'трансляции', 'насыпают', 'чудесное', 'комфортными', 'середину', 'глупо', 'лишнего']
['неприветливые', 'небольшую', 'звездные', 'кривые', 'ведёт', 'испорченное', 'среднюю', 'очередных', 'минутной']


In [330]:
tkn = Tokenizer(lower=True)
tkn.fit_on_texts(data.text.values)
data['seq_tokens'] = tkn.texts_to_sequences(data.text.values)
max_text_len = max(data.seq_tokens.apply(len))
fix_text_len = 294
print('Максимальная длина текста:',max_text_len,' слов')
data['seq_tokens'] = list(sequence.pad_sequences(data['seq_tokens'].values, maxlen=fix_text_len))
data.sample(2)

Максимальная длина текста: 294  слов


,raw_rating,raw_review,part,text,tokens,rating,seq_tokens
5419,1,\nОказались рядом и решили сходить семьей на «...,train,оказались рядом и решили сходить семьей на 007...,"[оказались, рядом, решили, сходить, семьей, 00...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5111,5,"\nрядом с домом,почти за углом; в тихом районч...",train,рядом с домом почти за углом в тихом райончике...,"[рядом, домом, углом, тихом, райончике, мало, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [331]:
test_x = data[data['part']=='test']['tokens'].apply(list).tolist()
train = data[data['part']=='train']

In [332]:
train_x, valid_x, train_y, valid_y = train_test_split(train['seq_tokens'].apply(list).tolist(),pd.get_dummies(train.rating.values), test_size=0.2)

In [333]:
num_features = max(GetFlatList(train.seq_tokens.apply(list).tolist()))
num_features

42315

In [335]:
batch_size = 32

model = Sequential()
model.add(Embedding(num_features+1, fix_text_len))
model.add(LSTM(100, return_sequences=True)) # new
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))

lr = 0.01

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              #optimizer=Adagrad(lr=lr),
              #optimizer=RMSprop(),
              metrics=['accuracy'])

filepath="weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

print('Train...')
early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=0.001, mode='auto')
model.fit(
    train_x,
    train_y.values,
    batch_size=batch_size,
    validation_data=(valid_x, valid_y.values),
    verbose=0,
    epochs=5,
    #callbacks=[TQDMNotebookCallback()]
    callbacks=[checkpoint, TQDMNotebookCallback(), reduce_lr]
)

model.save('Conv_Model.h5')

Train...


512/|/[loss: 0.063, acc: 0.982]   9%|| 512/5839 [00:27<01:38, 54.04it/s]                                               Epoch 00000: val_loss improved from inf to 0.37682, saving model to weights-improvement-00-0.3768.hdf5


Epoch 00001: val_loss did not improve


Epoch 00002: val_loss did not improve


Epoch 00003: val_loss did not improve


Epoch 00004: val_loss did not improve


In [337]:
test = data[data['part']=='test']#
pred = model.predict_proba(test['seq_tokens'].apply(list).tolist(),verbose=False)
test['preds'] = list(map(lambda x: 1 if x>=0.5 else -1, pred[:,1]))
test.head(10)

C:\Users\PDudukin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,raw_rating,raw_review,part,text,tokens,rating,seq_tokens,preds
7344,5,\nПо поводу бронирования по интернету- это кон...,test,по поводу бронирования по интернету это конечн...,"[поводу, бронирования, интернету, это, недочёт...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7345,3,\nне знаю как в остальных залах (думаю так же)...,test,не знаю как в остальных залах думаю так же но ...,"[знаю, остальных, залах, думаю, зале, номер, 5...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7346,5,"\nужасный кинотеатр! не понимаю, как в Ростове...",test,ужасный кинотеатр не понимаю как в ростове люд...,"[ужасный, кинотеатр, понимаю, ростове, люди, с...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7347,4,\nХороший кинотеатр!!!!!!!!!!!!!!!!!!!!!!!!!!!...,test,хороший кинотеатр,"[хороший, кинотеатр]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
7348,5,"\nМного разных залов, 1 и 2 вполне большие и у...",test,много разных залов 1 и 2 вполне большие и удоб...,"[разных, залов, 1, 2, вполне, большие, удобные...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7349,1,\nКупил билет через интернет (можно было выбра...,test,купил билет через интернет можно было выбрать ...,"[купил, билет, интернет, выбрать, места, схеме...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7350,5,\nНаш Российский народ настолько привык жалова...,test,наш российский народ настолько привык жаловать...,"[наш, российский, народ, настолько, привык, жа...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
7351,1,"\nУвидел информацию, что можно забронировать б...",test,увидел информацию что можно забронировать биле...,"[увидел, информацию, забронировать, билет, ста...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7352,1,\nВ кассе нельзя посмотреть какие билеты покуп...,test,в кассе нельзя посмотреть какие билеты покупае...,"[кассе, посмотреть, какие, билеты, покупаешь, ...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
7353,1,"\nОхранники тормазят еще хуже кассиров, билеты...",test,охранники тормазят еще хуже кассиров билеты за...,"[охранники, тормазят, хуже, кассиров, билеты, ...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1


In [338]:
score, acc = model.evaluate(valid_x, valid_y.values,batch_size=batch_size, verbose = 0)

In [339]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.958491744081
Test accuracy: 0.811643835453


In [342]:
model = load_model('TH_Model.h5')
model.load_weights("weights-improvement-01-0.3723.hdf5")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [343]:
def PredictAndColor(review):
    clean_review = ' '.join(re.split('\W+',review.lower()))
    #print(clean_review)
    tokens = [word for word in clean_review.split(' ') if word not in stop_words and word not in '']
    #print(tokens)
    seq_tokens = tkn.texts_to_sequences([clean_review])
    #print(seq_tokens)
    max_text_len = max(list(map(lambda x: len(x), seq_tokens)))
    seq_tokens = sequence.pad_sequences([seq_tokens[0]], maxlen=max_text_len)
    #print(seq_tokens)
    pred = model.predict(seq_tokens,verbose=False)
    if list(map(lambda x: 1 if x>=0.5 else -1, pred[:,1]))[0] == 1:
        word_list = [x for x in tokens if x in pos_list]
        return ColorWordInText(word_list, review, ['green'], 'expandonlycolor')
    else:
        word_list = [x for x in tokens if x in neg_list]
        return ColorWordInText(word_list, review, ['red'], 'expandonlycolor')

In [344]:
test_review_1 = 'непомерно дорогой кинотеатр, никогда больше не приду туда'
test_review_2 = 'Хороший кинотеатр с комфортными креслами'
error_review = 'В зале было холодно и довольно жесткие кресла'

review_id = random.randint(0,1000) 
print('ID:', review_id, '\nOriginal Review: ', data.raw_review[review_id])
PredictAndColor(data.raw_review[review_id])

ID: 540 
Original Review:  
Плохая система вентиляции, можно сказать никакая. Всегда очень душно, воздуха нет, после фильма выходишь с больной головой. 

